**About** : This notebook is used to train models.

In [ ]:
%load_ext autoreload
%autoreload 2

In [2]:
cd ../src/

/workspace/kaggle_rsna_abdominal/src


## Initialization

### Imports

In [3]:
import os
import torch

print(torch.__version__)
os.environ['CUDA_VISIBLE_DEVICES'] = "0"
device = torch.cuda.get_device_name(0)
print(device)

2.1.0a0+29c30b1
Tesla V100-SXM2-32GB-LS


In [4]:
import os
import sys
import glob
import json
import torch
import operator
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from tqdm import tqdm
from sklearn.metrics import *

In [5]:
from util.logger import (
    prepare_log_folder,
    save_config,
    create_logger,
#     init_neptune,
)

from params import *
from data.dataset import *
from data.preparation import *
from util.metrics import rsna_loss
from model_zoo.models_lvl2 import define_model

from training.main_lvl2 import k_fold, retrieve_preds

## Data

In [6]:
df_patient, df_img = prepare_data(DATA_PATH)

In [7]:
EXP_FOLDERS = [
    ("../logs/2023-09-06/4/", "seg"),
#     ("../logs/2023-09-15/2/", "probas"),
#     ("../logs/2023-09-15/11/", "probas"),
#     ("../logs/2023-09-15/22/", "probas"),
#     ("../logs/2023-09-15/37/", "probas"),
    ("../logs/2023-09-18/79/", "probas"),
    ("../logs/2023-09-18/78/", "probas"),
#     ("../logs/2023-09-18/77/", "probas"),
#     ("../logs/2023-09-18/76/", "probas"),
    ("../logs/2023-09-18/75/", "probas"),
    ("../logs/2023-09-18/74/", "probas"),
]
EXP_FOLDER = EXP_FOLDERS[0][0]

In [8]:
from inference.extract_features import Config
config = Config(json.load(open(EXP_FOLDER + "config.json", "r")))

In [9]:
if "fold" not in df_patient.columns:
    folds = pd.read_csv(config.folds_file)
    df_img = df_img.merge(folds)
    df_patient = df_patient.merge(folds)

In [10]:
dataset = PatientFeatureDataset(df_patient[df_patient['fold'] == 0], df_img[df_img['fold'] == 0], EXP_FOLDERS, max_len=1000)

In [11]:
fts, y, _ = dataset[0]
fts.size(), y

(torch.Size([1000, 93]), tensor([0., 0., 0., 0., 0.]))

In [12]:
# lens = []
# for i in tqdm(range(len(dataset))):
#     x = dataset[i][0]
#     lens.append(len(x))
    
# #     break

In [13]:
# sns.histplot(lens)

## Model

In [14]:
model = define_model("rnn", ft_dim=fts.size(-1), layer_dim=128, n_layers=2, dense_dim=256, num_classes=11, num_classes_aux=0)

In [15]:
x = torch.cat([fts.unsqueeze(0)] * 2)

pred, pred_aux = model(x)
pred.size()

torch.Size([2, 11])

## Training
- Handle variable sequence length more cleverly
- Tweak CNN
- Tweak fancier archs

In [68]:
class Config:
    """
    Parameters used for training
    """
    # General
    seed = 42
    verbose = 1
    device = "cuda"
    save_weights = True

    # Data
    exp_folders = [
        ("../logs/2023-09-06/4/", "seg"),
#         ("../logs/2023-09-15/22/", "probas"),
#         ("../logs/2023-09-15/37/", "probas"),
        ("../logs/2023-09-18/79/", "probas"),
#         ("../logs/2023-09-18/78/", "probas"),
    #     ("../logs/2023-09-18/77/", "probas"),
    #     ("../logs/2023-09-18/76/", "probas"),
#         ("../logs/2023-09-18/75/", "probas"),
#         ("../logs/2023-09-18/74/", "probas"),
    ]
    max_len = 1000
    n_fts = 0  # already pooled features, not supported yet

    # k-fold
    k = 4
    folds_file = f"../input/folds_{k}.csv"
    selected_folds = [0, 1, 2, 3]

    # Model
    name = "rnn"
    ft_dim = (11 + 11) * (len(exp_folders) - 1) + 5  # 1240 + 5

    dense_dim = 512
    layer_dim = 512
    n_layers = 1
    

    p = 0.
    use_msd = False
    num_classes = 11
    num_classes_aux = 0

    # Training    
    loss_config = {
        "name": "patient",
        "weighted": True,
        "use_any": True,
        "smoothing": 0,
        "activation": "patient",
        "aux_loss_weight": 0,
        "name_aux": "patient",
        "smoothing_aux": 0,
        "activation_aux": "",
    }

    data_config = {
        "batch_size": 64,
        "val_bs": 256,
        "mix": "mixup",
        "mix_proba": 0.,
        "sched": False,
        "mix_alpha": 4.,
        "additive_mix": False,
        "num_classes": num_classes,
        "num_workers": 8,
    }

    optimizer_config = {
        "name": "AdamW",
        "lr": 5e-4,  # 7e-4, 9e-4
        "warmup_prop": 0.,
        "betas": (0.9, 0.999),
        "max_grad_norm": 10.,
        "weight_decay": 0.2,
    }

    epochs = 10

    use_fp16 = True
    verbose = 1
    verbose_eval = 50

    fullfit = False
    n_fullfit = 1
    
    local_rank = 0
    distributed = False
    world_size = 1

In [69]:
DEBUG = True
log_folder = None

In [70]:
if not DEBUG:
    log_folder = prepare_log_folder(LOG_PATH)
    print(f"Logging results to {log_folder}")
    config_df = save_config(Config, log_folder + "config.json")
    create_logger(directory=log_folder, name="logs.txt")

preds, preds_aux = k_fold(Config, df_patient, df_img, log_folder=log_folder, run=None)


-------------   Fold 1 / 4  -------------

    -> 3557 training studies
    -> 1154 validation studies
    -> 4089867 trainable parameters

Epoch 01/10 (step 0050) 	lr=4.6e-04 	 t=7s 	 loss=0.789	 val_loss=0.666    rsna_loss=0.464
Epoch 02/10 (step 0100) 	lr=4.1e-04 	 t=6s 	 loss=0.625	 val_loss=0.607    rsna_loss=0.431
Epoch 03/10 (step 0150) 	lr=3.6e-04 	 t=6s 	 loss=0.579	 val_loss=0.582    rsna_loss=0.412
Epoch 04/10 (step 0200) 	lr=3.2e-04 	 t=6s 	 loss=0.567	 val_loss=0.620    rsna_loss=0.445
Epoch 05/10 (step 0250) 	lr=2.7e-04 	 t=6s 	 loss=0.560	 val_loss=0.565    rsna_loss=0.398
Epoch 06/10 (step 0300) 	lr=2.3e-04 	 t=6s 	 loss=0.532	 val_loss=0.569    rsna_loss=0.401
Epoch 07/10 (step 0350) 	lr=1.8e-04 	 t=6s 	 loss=0.552	 val_loss=0.562    rsna_loss=0.396
Epoch 08/10 (step 0400) 	lr=1.4e-04 	 t=6s 	 loss=0.538	 val_loss=0.563    rsna_loss=0.396
Epoch 09/10 (step 0450) 	lr=9.2e-05 	 t=6s 	 loss=0.531	 val_loss=0.566    rsna_loss=0.399
Epoch 10/10 (step 0500) 	lr=4.6e-05 	 t=

### Eval

In [42]:
LOG_FOLDER = "../logs/2023-09-15/33/"  # 0.419 - v2s
LOG_FOLDER = "../logs/2023-09-15/36/"  # 0.407 - v2s aug4

LOG_FOLDER = "../logs/2023-09-18/81/"  # 0.388 - b5
# LOG_FOLDER ="../logs/2023-09-18/82/"  # 0.394 - b5
# LOG_FOLDER ="../logs/2023-09-18/84/"  # 0.402 - v2s

In [38]:
!cat $LOG_FOLDER/logs.txt


-------------   Fold 1 / 4  -------------

    -> 3557 training studies
    -> 1154 validation studies
    -> 5795851 trainable parameters

Epoch 01/10 (step 0050) 	lr=4.6e-04 	 t=9s 	 loss=0.775	 val_loss=0.681    rsna_loss=0.491
Epoch 02/10 (step 0100) 	lr=4.1e-04 	 t=7s 	 loss=0.627	 val_loss=0.587    rsna_loss=0.407
Epoch 03/10 (step 0150) 	lr=3.6e-04 	 t=7s 	 loss=0.578	 val_loss=0.568    rsna_loss=0.397
Epoch 04/10 (step 0200) 	lr=3.2e-04 	 t=7s 	 loss=0.571	 val_loss=0.564    rsna_loss=0.397
Epoch 05/10 (step 0250) 	lr=2.7e-04 	 t=7s 	 loss=0.567	 val_loss=0.560    rsna_loss=0.386
Epoch 06/10 (step 0300) 	lr=2.3e-04 	 t=7s 	 loss=0.566	 val_loss=0.568    rsna_loss=0.401
Epoch 07/10 (step 0350) 	lr=1.8e-04 	 t=7s 	 loss=0.531	 val_loss=0.583    rsna_loss=0.411
Epoch 08/10 (step 0400) 	lr=1.4e-04 	 t=7s 	 loss=0.556	 val_loss=0.562    rsna_loss=0.396
Epoch 09/10 (step 0450) 	lr=9.2e-05 	 t=7s 	 loss=0.532	 val_loss=0.559    rsna_loss=0.393
Epoch 10/10 (step 0500) 	lr=4.6e-05 	 t=

In [33]:
df_oof, pred_oof = retrieve_preds(df_patient, df_img, Config, LOG_FOLDER)

In [34]:
losses, avg_loss = rsna_loss(pred_oof, df_oof)

for k, v in losses.items():
    print(f"- {k.split('_')[0][:8]} loss\t: {v:.3f}")
    
print(f'\n -> CV Score : {avg_loss :.3f}')

- bowel loss	: 0.129
- extravas loss	: 0.525
- kidney loss	: 0.301
- liver loss	: 0.450
- spleen loss	: 0.468
- any loss	: 0.454

 -> CV Score : 0.388


In [23]:
# pred_oof_ = pred_oof.copy()
# losses, avg_loss = rsna_loss(pred_oof, df_oof)
# best_score = avg_loss

# for _ in range(2):
#     factors = []
#     for i in range(pred_oof.shape[1]):
#         scores = {}
#         for factor in np.round(np.arange(0.5, 1.5, 0.1), 2):
#             for shift in np.round(np.arange(-0.1, 0.11, 0.1), 2):
# #             for shift in [-0.1, 0, 0.1]:
#                 pred_oof_r = pred_oof_.copy()
#                 pred_oof_r[:, i] = pred_oof_r[:, i] * factor + shift
#                 pred_oof_r[:, i] = np.clip(pred_oof_r[:, i], 0.00001, 0.99999)

#                 losses, avg_loss = rsna_loss(pred_oof_r, df_oof)
#                 scores[(factor, shift)] = avg_loss

#     #     print(scores)
#         best_coefs, best_loss = min(scores.items(), key=operator.itemgetter(1))
#         pred_oof_[:, i] = np.clip(pred_oof_[:, i] * best_coefs[0] + best_coefs[1], 0.00001, 0.99999)
#         best_score = best_loss
#         print(f'{i} - {best_coefs}  -  {best_loss :.3f}')
#         factors.append(best_coefs)

In [24]:
dummy = np.array(
    [
        [0.04] * len(df_oof),
        [0.3] * len(df_oof),
        [0.6] * len(df_oof), [0.05] * len(df_oof), [0.05] * len(df_oof),
        [0.4] * len(df_oof), [0.07] * len(df_oof), [0.03] * len(df_oof),
        [0.3] * len(df_oof), [0.04] * len(df_oof), [0.07] * len(df_oof),
    ]
).T
losses, avg_loss = rsna_loss(dummy, df_oof)

for k, v in losses.items():
    print(f"- {k.split('_')[0][:8]} loss\t: {v:.3f}")
    
print(f'\n -> CV Score : {avg_loss :.3f}')

- bowel loss	: 0.168
- extravas loss	: 0.602
- kidney loss	: 0.511
- liver loss	: 0.653
- spleen loss	: 0.760
- any loss	: 0.618

 -> CV Score : 0.552


In [ ]:
# for i in range(2):
#     sns.histplot(preds[:, i])
    
#     auc = roc_auc_score(df_val[PATIENT_TARGETS[i]], preds[:, i])
#     print(f'- {PATIENT_TARGETS[i]} auc : {auc:.3f}')
    
#     plt.show()

Done ! 